In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import getpass
import pdvega
import time
from configobj import ConfigObj

In [2]:
# Create a posgresql database connection using settings from config file 

# Create a database connection using settings from config file
config='../db/config.ini'
# connection info
conn_info = dict()
if os.path.isfile(config):
    config = ConfigObj(config)
    conn_info["sqluser"] = config['username']
    conn_info["sqlpass"] = config['password']
    conn_info["sqlhost"] = config['host']
    conn_info["sqlport"] = config['port']
    conn_info["dbname"] = config['dbname']
    conn_info["schema_name"] = config['schema_name']
else:
    conn_info["sqluser"] = 'postgres'
    conn_info["sqlpass"] = '243'
    conn_info["sqlhost"] = 'localhost'
    conn_info["sqlport"] = 5432
    conn_info["dbname"] = 'eicu'
    conn_info["schema_name"] = 'public,eicu'
    
# Connect to the eICU database
print('Database: {}'.format(conn_info['dbname']))
print('Username: {}'.format(conn_info["sqluser"]))
if conn_info["sqlpass"] == '243':
    # try connecting without password, i.e. peer or OS authentication
    try:
        if (conn_info["sqlhost"] == 'localhost') & (conn_info["sqlport"]=='5432'):
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                    user=conn_info["sqluser"])            
        else:
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                    host=conn_info["sqlhost"],
                                    port=conn_info["sqlport"],
                                    user=conn_info["sqluser"])
    except:
        conn_info["sqlpass"] = getpass.getpass('Password: ')
    
        con = psycopg2.connect(dbname=conn_info["dbname"],
                                host=conn_info["sqlhost"],
                                port=conn_info["sqlport"],
                                user=conn_info["sqluser"],
                                password=conn_info["sqlpass"])
query_schema = 'set search_path to ' + conn_info['schema_name'] + ';'

Database: eicu
Username: postgres
Password: ········


In [3]:
#import patient data, if not exsit allpatientdata.csv file than please excute the patientdata.ipynb file first
df=pd.read_csv(os.getcwd()+'\\allpatientfile.csv')

In [4]:
def main():
    start = time.clock()
    #4 kinds of medicine we want to extract
    ftName_0 = ['Vasopressor', 'Relaxant', 'Sedation', 'PPI']
    ftName = ['patientunitstayid']
    for i in range(0,len(ftName_0)):
        for j in range(0,30):
            ftName.append('%s D%02d'%(ftName_0[i], j+1))
    df_IFD = pd.DataFrame(columns= ftName)
    th = 0
    for i in range(0,len(df)):
        print("%d/%d: %s"%(i,len(df),df['patientunitstayid'][i]),end='\r')
        df_IFD = Infusiondrug(df['patientunitstayid'][i], df['Firstday'][i], df['LoS'][i], th, df_IFD, ftName_0)
        th =th+1
    df_IFD.to_csv(os.getcwd()+'\\infusiondrug.csv',index=False)
    end = time.clock() - start
    print(end)

In [5]:
#可調整輸出的方式
#adjust the extract data into output format
def Featrue_all_value_with_day(ft_name, day, df_infusiondrug):
    temp_1 = [0]
    for j in range(0,len(df_infusiondrug.index)):
        if df_infusiondrug['drugname'][j] == ft_name:  
                if df_infusiondrug['date'][j] == day: 
                    if df_infusiondrug['drugrate'][j] != 0:
                        df_infusiondrug['drugrate'][j] = 1
                        temp_1.append(df_infusiondrug['drugrate'][j])
    if len(temp_1) != 1:
        temp_1.pop(0)
    return np.max(temp_1)


In [6]:
def Infusiondrug(patientunitstayid, Firstday, lenofstay, th, df_IFD, ftName_0):
    query = query_schema + """
    select *
    from infusiondrug
    where patientunitstayid = '{}'
    order by infusionoffset
    """.format(int(patientunitstayid))

    df_infusiondrug = pd.read_sql_query(query, con)
    df_infusiondrug = df_infusiondrug[['patientunitstayid','infusionoffset','drugname','drugrate']]

    date = np.zeros((len(df_infusiondrug.index),), dtype=int)
    df_infusiondrug['date'] = date
    #calculate event time by days
    for i in range(0,len(df_infusiondrug.index)):
        temp = df_infusiondrug['infusionoffset'][i]
        if temp < Firstday:
            df_infusiondrug['date'][i] = 1
        elif temp >= Firstday:
            temp = temp - Firstday
            df_infusiondrug['date'][i] = temp // (24*60) + 2
    ftName = ['patientunitstayid']

    if df_infusiondrug.size == 0:
        space_row = [patientunitstayid]
        for i in range(len(ftName_0)):
            for j in range(0,30):
                if j >= int(lenofstay):
                    space_row.append(0)
                else:
                    space_row.append(0)
        df_IFD = df_IFD.append(pd.Series(space_row, index=df_IFD.columns), ignore_index=True)
    else:
        space_row = [patientunitstayid]
        for i in range(len(ftName_0)):
            for j in range(0,30):
                if j >= int(lenofstay):
                    space_row.append(0)
                else:
                    space_row.append(0)
        df_IFD = df_IFD.append(pd.Series(space_row, index=df_IFD.columns), ignore_index=True)
        #choose the madicine we want and turn it into same class
        for i in range(0,len(df_infusiondrug.index)):
            temp = df_infusiondrug['drugname'][i].lower().strip(' ')

            if(temp==''):
                continue
            else:
                temp = temp.split()[0]

                if temp == 'norepinephrine' or temp == 'epinephrine' or temp == 'dopmaine' or temp == 'vasopressin':
                    df_infusiondrug['drugname'][i] = 'Vasopressor'
                elif temp == 'nimbex' or temp == 'vecuronium':
                    df_infusiondrug['drugname'][i] = 'Relaxant'
                elif temp == 'midazolam' or temp == 'propofol' or temp == 'precedex' or temp == 'dexmedetomidine':
                    df_infusiondrug['drugname'][i] = 'Sedation'        
                elif temp == 'esomeprazole' or temp =='pantoprazole':
                    df_infusiondrug['drugname'][i] = 'PPI'
                elif temp =='fentanyl':
                    df_infusiondrug['drugname'][i] = 'Pain control'
                else:
                    continue
    
        #將計算好的值放入表格中  
        #put the calculated value into sheet and call "Feature_all_value_with_day" function to turning it into specific format
        for i in range(0,len(ftName_0)):
            for j in range(0,30):
                if j >= int(lenofstay):
                    break
                else:
                    str_temp = '%s D%02d'%(ftName_0[i],j+1)
                    df_IFD[str_temp][th] = Featrue_all_value_with_day(ftName_0[i],j+1,df_infusiondrug)
    return df_IFD

In [7]:
if __name__ is '__main__':
    main()

c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_lau

c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1320.9641343353226.0


c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  app.launch_new_instance()
